# Main Focus of the Notebook: Running Laptrack on detections

### This notebook takes as input the dataframe which contains filtered detected spots and runs laptrack module on it. 
### A new dataframe containing tree id and track id is returned. For us main variable for tracks is track id. 

In [1]:
from os import path
import pandas as pd
from IPython.display import display
from matplotlib import pyplot as plt
from laptrack import LapTrack, ParallelBackend
import napari
from skimage import io
import time  
import os 
import sys

### Do not change the code in the cell below

In [2]:
# This assumes that your notebook is inside 'Jupyter Notebooks', which is at the same level as 'test_data'
base_dir = os.path.join(os.path.dirname(os.path.abspath("__file__")), '..', 'test_data')

zarr_directory = 'zarr_file/all_channels_data'
zarr_full_path = os.path.join(base_dir, zarr_directory)

input_directory = 'datasets'
input_file_name = 'cleaned_spots_intensities_c3_all.pkl'
input_directory_full = os.path.join(base_dir,input_directory, input_file_name)

output_directory = 'datasets'
output_file_name = 'track_df_c3_cleaned.pkl'
output_directory_full = os.path.join(base_dir,output_directory, output_file_name)

In [3]:
cleaned_spots_df = pd.read_pickle(input_directory_full)

In [4]:
max_distance = 3
lt = LapTrack(
    track_dist_metric="sqeuclidean",  # The similarity metric for particles. See `scipy.spatial.distance.cdist` for allowed values.
    splitting_dist_metric="sqeuclidean",
    merging_dist_metric="sqeuclidean",
    # the square of the cutoff distance for the "sqeuclidean" metric
    track_cost_cutoff=max_distance**2,
    gap_closing_cost_cutoff = (2*max_distance)**2,
    gap_closing_max_frame_count = 1,
    splitting_cost_cutoff=False,  # or False for non-splitting case
    merging_cost_cutoff=False,  # or False for non-merging case
)

In [5]:
start_time = time.time()

track_df, split_df, merge_df = lt.predict_dataframe(
    cleaned_spots_df,
    coordinate_cols=[
        "mu_x",
        "mu_y",
        "mu_z"
    ],  # the column names for the coordinates
    frame_col="frame",  # the column name for the frame (default "frame")
    only_coordinate_cols=False,  # if False, returned track_df includes columns not in coordinate_cols.
    # False will be the default in the major release.
)

end_time = time.time()

execution_time = end_time - start_time
print(f"Total execution time: {execution_time} seconds")
track_df = track_df.reset_index()

Total execution time: 97.6930239200592 seconds


In [6]:
# Save the DataFrame to a pickle file
track_df.to_pickle(output_directory_full)